## **Mengimport Library yang dibutuhkan**

In [1]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import math
import os
import csv

In [16]:
labels = ['celana_panjang', 'celana_pendek', 'Kaos']
imgPerLabels = 200
orientation = [0, 45, 90, 135]
datasets = {}
input_folder = 'dataset_closed/'

for j in range(len(labels)):
    datasets[labels[j]] = np.empty((imgPerLabels, 256, 256))
    count=0
        # if os.path.isdir(labels[j]+"\\"+str(imgindex)):
    for filename in os.listdir(input_folder):
        if(labels[j] in filename):
            img_path = os.path.join(input_folder, filename)
            img = cv.cvtColor(cv.resize(cv.imread(img_path), (256, 256)), cv.COLOR_BGR2GRAY)/255
            datasets[labels[j]][count] = img
            count+=1
            datasets[labels[j]][count] = cv.rotate(img, cv.ROTATE_180)
            count+=1
        if count==imgPerLabels: break

Code di atas memiliki tujuan untuk membuat dataset yang berisi gambar-gambar dengan label-label tertentu dari folder `dataset_closed`. Dataset akan dibentuk berdasarkan jumlah gambar per label (`imgPerLabels`) dan orientasi gambar yang akan dibentuk (`orientation`). Dataset tersebut disimpan dalam variabel `datasets`.

Pertama, kita mendefinisikan label-label yang akan digunakan dalam dataset (`labels`) dan jumlah gambar per label (`imgPerLabels`). Kita juga mendefinisikan daftar orientasi gambar yang akan dibentuk (`orientation`). Selanjutnya, kita membuat dictionary `datasets` untuk menyimpan dataset dengan label sebagai kunci dan array kosong dengan dimensi `(imgPerLabels, 256, 256)` sebagai nilai awalnya. Selanjutnya, dilakukan iterasi pada setiap label menggunakan loop `for`. Di dalam loop tersebut, kita menginisialisasi variabel `count` sebagai penghitung jumlah gambar yang telah ditambahkan ke dataset.

Selanjutnya, dilakukan iterasi pada setiap file dalam folder `input_folder`. Dalam setiap iterasi, dilakukan pengecekan apakah label terdapat dalam nama file menggunakan `if(labels[j] in filename)`. Jika label terdapat dalam nama file, maka gambar tersebut akan dibaca, diubah ukurannya menjadi 256x256 menggunakan `cv.resize()`, diubah menjadi grayscale, dan dinormalisasi ke rentang 0 hingga 1. Selanjutnya, gambar tersebut ditambahkan ke dataset dengan label yang sesuai dan indeks `count`. Setelah itu, gambar tersebut dirotasi sebesar 180 derajat menggunakan `cv.rotate()` dan ditambahkan ke dataset dengan indeks `count+1`.

Proses ini dilakukan hingga jumlah gambar yang telah ditambahkan ke dataset mencapai `imgPerLabels` atau hingga semua file dalam folder telah diiterasi. Jika jumlah gambar yang telah ditambahkan mencapai `imgPerLabels`, maka loop dihentikan menggunakan `break`. Setelah loop selesai dieksekusi untuk satu label, dataset tersebut akan tersimpan dalam dictionary `datasets`. Proses ini diulangi untuk setiap label yang terdapat dalam `labels`, sehingga setelah loop selesai dieksekusi, dataset lengkap dengan label dan gambar-gambar yang telah diolah akan tersedia dalam `datasets`.

In [17]:
def getGlcm(datasets:np.ndarray, ori:np.ndarray[int, int], dist=1):
    result = {}
    
    for i in datasets:
        print(i)
        label = []
        for j in range(len(datasets[i])):
            img:np.ndarray = datasets[i][j]
            imgGlcm = {}
            for k in range(len(ori)):
                (iH, iW) = img.shape[:2]
                matriks = np.zeros((256, 256), float)
                startX = startY = 0
                endX = iW
                endY = iH
                
                if ori[k]==0: endX=iW-dist
                elif ori[k]==90: startY=dist
                elif ori[k]==45:
                    startY=dist
                    endX=iW-dist
                elif ori[k]==135:
                    startY=startX=dist
                
                for y in range(startY, endY):
                    for x in range(startX, endX):
                        if ori[k]==0: matriks[int(img[y,x]), int(img[y,x+dist])] =+ 1
                        elif ori[k]==90: matriks[int(img[y,x]), int(img[y-dist,x])] =+ 1
                        elif ori[k]==45: matriks[int(img[y,x]), int(img[y-dist,x+dist])] =+ 1
                        elif ori[k]==135: matriks[int(img[y,x]), int(img[y-dist,x-dist])] =+ 1

                matriks +=matriks.transpose()
                summary = np.sum(matriks)
                matriks /= summary
                # for y in range(matriks.shape[0]):
                #     for x in range(matriks.shape[1]):
                #         if matriks[y,x] != 0: matriks[y,x] /= summary
                
                imgGlcm[orientation[k]] = matriks
            label.append(imgGlcm)
        result[i] = label
        print(i)
    return result

glcm = getGlcm(datasets, orientation)

celana_panjang
celana_panjang
celana_pendek


Fungsi `getGlcm` digunakan untuk menghitung matriks GLCM (Gray-Level Co-occurrence Matrix) dari dataset gambar yang diberikan, dengan orientasi dan jarak yang ditentukan. Hasil perhitungan GLCM disimpan dalam bentuk dictionary dengan label sebagai kunci dan daftar matriks GLCM untuk setiap gambar sebagai nilai. Pada awal fungsi, sebuah dictionary kosong `result` dibuat untuk menyimpan hasil perhitungan GLCM. Selanjutnya, dilakukan iterasi pada setiap label dalam dataset menggunakan loop `for`. Di dalam loop tersebut, dilakukan iterasi pada setiap gambar dalam label tersebut menggunakan loop `for`.

Dalam setiap iterasi, gambar `img` diambil dari dataset. Kemudian, untuk setiap orientasi dalam `ori`, matriks GLCM `matriks` yang berukuran 256x256 diinisialisasi dengan nilai 0. Selanjutnya, berdasarkan orientasi, dilakukan penentuan batas awal dan akhir untuk melakukan iterasi pada piksel gambar. Jika orientasi adalah 0, batas akhir kolom dikurangi dengan jarak `dist`. Jika orientasi adalah 90, batas awal baris ditambah dengan jarak `dist`. Jika orientasi adalah 45, batas awal baris ditambah dengan jarak `dist` dan batas akhir kolom dikurangi dengan jarak `dist`. Jika orientasi adalah 135, batas awal baris dan kolom ditambah dengan jarak `dist`.

Selanjutnya, dilakukan iterasi pada piksel-piksel gambar dalam batas yang ditentukan. Dalam setiap iterasi, dilakukan penambahan frekuensi kemunculan pasangan nilai piksel ke matriks GLCM berdasarkan orientasi yang sesuai. Setelah selesai iterasi pada piksel, matriks GLCM dijumlahkan dengan transpose-nya untuk menghasilkan matriks simetris. Kemudian, dilakukan normalisasi matriks dengan membaginya dengan jumlah total elemen dalam matriks. Matriks GLCM yang telah dihitung kemudian disimpan dalam dictionary `imgGlcm` dengan orientasi sebagai kunci.

Selanjutnya, matriks GLCM untuk setiap gambar dalam label disimpan dalam daftar `label`. Setelah selesai iterasi pada gambar-gambar dalam label, daftar `label` tersebut disimpan dalam dictionary `result` dengan label sebagai kunci. Terakhir, fungsi mengembalikan dictionary `result` yang berisi hasil perhitungan GLCM untuk setiap label dan gambar. Seluruh proses tersebut dijalankan dengan memanggil fungsi `getGlcm` pada variabel `glcm` dengan menggunakan dataset `datasets` dan orientasi `orientation`.

In [ ]:
def getFeature(glcm:dict[int, dict[int, np.ndarray]], features:list[str]=['contrast', 'dissimiliarity', 'homogeneity', 'entropy', 'asm', 'corellation', 'energy']):
    
    result = []
    for i in glcm.keys():
        for j in range(len(glcm[i])):
            imgFeatures = {}
            for k in glcm[i][j].keys():
                matriks = glcm[i][j][k]
                
                fitur = {f'contrast{k}':0.0, 
                            f'dissimiliarity{k}':0.0, 
                            f'homogeneity{k}':0.0, 
                            f'entropy{k}':0.0, 
                            f'asm{k}':0.0, 
                            f'corellation{k}':0.0, 
                            f'energy{k}':0.0}
                
                meanY = np.mean(matriks)
                meanX = np.mean(matriks.transpose())
                
                for y in range(256):
                    for x in range(256):
                        if matriks[y,x]==0.0: continue
                        if "contrast" in features: fitur[f"contrast{k}"] += matriks[y,x]*math.pow(y-x, 2)
                        if "dissimiliarity" in features: fitur[f"dissimiliarity{k}"] += matriks[y,x]*abs(y-x)
                        if "homogeneity" in features: fitur[f"homogeneity{k}"] += matriks[y,x]/(1+math.pow(y-x, 2))
                        if "entropy" in features: fitur[f"entropy{k}"] -= matriks[y,x]*math.log10(matriks[y,x])
                        if "asm" in features: fitur[f"asm{k}"] += math.pow(matriks[y,x], 2)
                
                if "corellation" in features:
                    stdX = np.std(matriks)
                    stdY = np.std(matriks.transpose())
                    if stdX != 0 and stdY != 0: fitur[f"corellation{k}"] = (- (meanX * meanY)) / (stdX * stdY)
                if 'energy' in features: fitur[f'energy{k}'] = math.sqrt(fitur[f"asm{k}"])
                
                for l in fitur:
                    imgFeatures[l] = fitur[l]
            
            imgFeatures['label'] = i        
            result.append(imgFeatures)
    return result
features = getFeature(glcm)

Fungsi `getFeature` memiliki tujuan untuk menghitung berbagai fitur statistik berdasarkan matriks GLCM (Gray Level Co-occurrence Matrix) yang telah dihitung sebelumnya. Fungsi ini menerima dua parameter, yaitu `glcm` yang merupakan sebuah dictionary berisi matriks GLCM untuk setiap label dan gambar, serta `features` yang merupakan sebuah daftar fitur yang ingin dihitung. 

Pertama-tama, sebuah list kosong `result` dibuat untuk menampung hasil perhitungan fitur dari setiap gambar. Selanjutnya, dilakukan iterasi pada setiap label dalam `glcm` menggunakan loop `for i in glcm.keys()`. Dalam setiap iterasi, dilakukan iterasi pada setiap gambar dalam label tersebut menggunakan loop `for j in range(len(glcm[i]))`. Di dalam loop gambar, sebuah dictionary `imgFeatures` dibuat untuk menyimpan hasil perhitungan fitur dari gambar tersebut. Kemudian, dilakukan iterasi pada setiap orientasi dalam matriks GLCM untuk gambar tersebut menggunakan loop `for k in glcm[i][j].keys()`. Dalam setiap iterasi, matriks GLCM `matriks` diambil dari `glcm` berdasarkan label dan gambar yang sesuai.

Selanjutnya, sebuah dictionary `fitur` dibuat untuk menyimpan hasil perhitungan fitur dari matriks GLCM tersebut. Setiap fitur yang ingin dihitung diberikan nilai awal 0.0. Selanjutnya, rata-rata `meanY` dan `meanX` dari matriks GLCM dihitung menggunakan fungsi `np.mean()`. Rata-rata ini akan digunakan dalam perhitungan beberapa fitur seperti "corellation". Setelah itu, dilakukan iterasi pada setiap elemen dalam matriks GLCM menggunakan nested loop `for y in range(256)` dan `for x in range(256)`. Dalam setiap iterasi, fitur-fitur yang sesuai dengan `features` dihitung berdasarkan rumus-rumus yang telah ditentukan. Rumus yang digunakan melibatkan perhitungan seperti perbedaan intensitas, kuadrat perbedaan intensitas, nilai absolut perbedaan intensitas, dan logaritma dari elemen matriks GLCM. Nilai-nilai fitur ini diakumulasikan ke dalam `fitur` sesuai dengan formula yang sesuai.

Setelah selesai iterasi pada elemen-elemen matriks GLCM, fitur-fitur yang bergantung pada rata-rata dan standar deviasi, seperti "corellation" dan "energy", dihitung menggunakan rumus-rumus yang telah ditentukan. Fitur "corellation" membutuhkan perhitungan standar deviasi dari matriks GLCM, yang dihitung menggunakan fungsi `np.std()`. Fitur "energy" dihitung sebagai akar kuadrat dari fitur "asm" (Angular Second Moment). Selanjutnya, dilakukan iterasi pada setiap fitur dalam `fitur` menggunakan loop `for l in fitur`. Setiap fitur tersebut disimpan dalam dictionary `imgFeatures`. Setelah selesai iterasi pada orientasi, fitur-fitur dari gambar tersebut telah dihitung dan disimpan dalam `imgFeatures`. Kemudian, label gambar ditambahkan ke `imgFeatures` dengan kunci 'label'.

Hasil perhitungan fitur untuk gambar tersebut disimpan dalam `result` sebagai elemen list. Setelah selesai iterasi pada gambar-gambar dalam label, hasil perhitungan fitur untuk setiap gambar dalam setiap label telah disimpan dalam `result`. Terakhir, fungsi mengembalikan `result` yang berisi hasil perhitungan fitur untuk semua gambar dalam setiap label.

In [ ]:
header = []

for head in features[0].keys():
    header.append(head)

with open('Features.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(features)

Code di atas bertujuan untuk menyimpan hasil perhitungan fitur dalam format CSV. Pertama, sebuah list kosong `header` dibuat untuk menyimpan nama-nama kolom dalam file CSV. Selanjutnya, dilakukan iterasi pada keys dari elemen pertama dalam `features` menggunakan loop `for head in features[0].keys()`. Dalam setiap iterasi, nama kolom `head` ditambahkan ke list `header`. Setelah itu, dibuka file CSV dengan nama 'Features.csv' menggunakan `open()` dengan mode 'w' untuk menulis, dan `newline=''` untuk menghindari penambahan baris kosong di antara baris data. Selanjutnya, objek `writer` dibuat menggunakan `csv.DictWriter` dengan parameter file yang telah dibuka (`f`) dan fieldnames (`header`). Hal ini digunakan untuk menulis data sebagai dictionary ke file CSV.

Pada tahap ini, header baris ditulis ke file CSV menggunakan method `writeheader()` pada objek `writer`. Selanjutnya, data fitur ditulis ke file CSV menggunakan method `writerows()` pada objek `writer`. Data yang ditulis adalah `features`, yang merupakan list of dictionaries yang berisi hasil perhitungan fitur.Terakhir, setelah selesai penulisan, file CSV ditutup.